In [2]:
import pandas as pd
import numpy as np
import requests
from pyproj import Transformer

df = pd.read_csv('../data/conus_ak/fired_conus_ak_2000_to_2024_events.csv')

transformer = Transformer.from_crs("epsg:32610", "epsg:4326", always_xy=True)  # Adjust EPSG if UTM zone varies
df['lon'], df['lat'] = transformer.transform(df['ig_utm_x'].values, df['ig_utm_y'].values)

df['ig_date'] = pd.to_datetime(df['ig_date'])
df['last_date'] = pd.to_datetime(df['last_date'])

def get_event_dates(row):
    return [row['ig_date'] + pd.Timedelta(days=i) for i in range(int(row['event_dur']))]

df['event_dates'] = df.apply(get_event_dates, axis=1)

df.head()

,id,geometry,ig_date,ig_day,ig_month,ig_year,last_date,event_dur,tot_pix,tot_ar_km2,...,lc_name,lc_desc,lc_type,eco_mode,eco_name,eco_type,tot_perim,lon,lat,event_dates
0,1,MULTIPOLYGON (((-10010798.656358264 4834667.34...,2007-02-01,32,2,2007,2007-02-01,1,1,0.214659,...,Evergreen Needleleaf Forests,Dominated by evergreen conifer trees (canopy>2...,IGBP global vegetation classification scheme,10.0,Central Pacific coastal forests,WWF Terrestrial Ecoregions of the World,1861.250866,163.280081,14.680764,[2007-02-01 00:00:00]
1,3,MULTIPOLYGON (((-10013578.656358264 4818451.34...,2003-10-31,304,10,2003,2003-11-04,5,5,1.073293,...,Evergreen Needleleaf Forests,Dominated by evergreen conifer trees (canopy>2...,IGBP global vegetation classification scheme,10.0,Central Pacific coastal forests,WWF Terrestrial Ecoregions of the World,7419.250866,163.312547,14.634074,"[2003-10-31 00:00:00, 2003-11-01 00:00:00, 200..."
2,4,MULTIPOLYGON (((-10026087.656358264 4817061.34...,2007-02-02,33,2,2007,2007-02-02,1,2,0.429317,...,Evergreen Needleleaf Forests,Dominated by evergreen conifer trees (canopy>2...,IGBP global vegetation classification scheme,10.0,Central Pacific coastal forests,WWF Terrestrial Ecoregions of the World,2787.250866,163.281524,14.602774,[2007-02-02 00:00:00]
3,5,MULTIPOLYGON (((-10025624.656358264 4816598.34...,2013-10-30,303,10,2013,2013-10-30,1,1,0.214659,...,Evergreen Needleleaf Forests,Dominated by evergreen conifer trees (canopy>2...,IGBP global vegetation classification scheme,10.0,Central Pacific coastal forests,WWF Terrestrial Ecoregions of the World,1861.250866,163.283768,14.602657,[2013-10-30 00:00:00]
4,6,MULTIPOLYGON (((-10024234.656358264 4816134.84...,2013-10-30,303,10,2013,2013-10-30,1,1,0.214659,...,Evergreen Needleleaf Forests,Dominated by evergreen conifer trees (canopy>2...,IGBP global vegetation classification scheme,10.0,Central Pacific coastal forests,WWF Terrestrial Ecoregions of the World,1861.250866,163.288380,14.604608,[2013-10-30 00:00:00]


In [3]:
df_filtered = df[df['ig_date'] > '2016-01-01'].copy()
len(df_filtered)

122481

In [ ]:
import json
import requests
import numpy as np
import pandas as pd
import time

OPEN_MATEO_API_KEY = "0VsL87d9nCTz6eRe"

weather_variables = [
  'mean_temperature_2m', 'max_temperature_2m',
  'mean_relative_humidity_2m', 'min_relative_humidity_2m',
  'precipitation_sum', 'precipitation_hours', 'rain_sum',
  'et0_fao_evapotranspiration', 'vapour_pressure_deficit_max',
  'mean_wind_speed_10m', 'max_wind_speed_10m',
  'mean_wind_gusts_10m', 'max_wind_gusts_10m',
  'dominant_wind_direction_10m',
  'shortwave_radiation_sum', 'leaf_wetness_probability_mean'
]


def get_weather(lat, lon, ignition_date):
    # url = "https://archive-api.open-meteo.com/v1/archive"
    # url = "https://customer-archive-api.open-meteo.com/v1/archive"
    url = "https://customer-api.open-meteo.com/v1/forecast"

    preignition_date = (pd.to_datetime(ignition_date) - pd.Timedelta(days=14)).strftime('%Y-%m-%d')
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": preignition_date,
        "end_date": ignition_date,
        "daily": weather_variables,
        "timezone": "auto",
        "apikey": OPEN_MATEO_API_KEY
    }
    response = requests.get(url, params=params, timeout=10)

    if response.status_code == 200:
        data = response.json()
        daily = data['daily']

        # Use the first day of returned data
        idx = 0
        augmented_data = {
            'tavg': daily['temperature_2m_mean'][idx],
            'prcp': daily['precipitation_sum'][idx],
            'wspd': daily['wind_speed_10m_max'][idx],
            'wdir': daily['wind_direction_10m_dominant'][idx],
            'wpgt': daily['wind_gusts_10m_max'][idx],
            'pres_max': daily.get('pressure_msl_max', [None])[idx],
            'pres_min': daily.get('pressure_msl_min', [None])[idx],
            'tsun': daily.get('sunshine_duration', [None])[idx],
            'relative_humidity_2m_mean': daily.get('relative_humidity_2m_mean', [None])[idx]
        }

        return augmented_data
    else:
        print(f"Error fetching weather data: HTTP {response.status_code}, {response.text}")
        return None

def enrich_row(row):
    start = row['ig_date'].strftime('%Y-%m-%d')
    end = row['last_date'].strftime('%Y-%m-%d')
    weather_data = get_weather(row['lat'], row['lon'], start, end)
    if weather_data:
        # print(f"Getting weather info for {start}")
        # print(f"Weather data: {json.dumps(weather_data, indent=2)}")
        return pd.Series(weather_data)
    else:
        return pd.Series({key: None for key in [
            'tavg', 'prcp', 'wspd', 'wdir', 'wpgt', 'pres_max', 'pres_min', 'tsun', 'relative_humidity_2m_mean'
        ]})


In [ ]:
from tqdm import tqdm

tqdm.pandas()

# Apply the enrichment function to each row
weather_df = df_filtered.progress_apply(enrich_row, axis=1)

# Concatenate weather data to the original DataFrame
df_filtered = pd.concat([df_filtered, weather_df], axis=1)
df_filtered.head()

  0%|          | 359/122481 [07:51<44:31:33,  1.31s/it]


ReadTimeout: HTTPSConnectionPool(host='customer-api.open-meteo.com', port=443): Read timed out. (read timeout=10)

In [ ]:
df_filtered.to_csv('fired_conus_with_weather.csv', index=False)

# BAH


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('fired_conus_ak_2000_to_2024_events.csv')

In [ ]:
from pyproj import Transformer

transformer = Transformer.from_crs("epsg:32610", "epsg:4326", always_xy=True)  # Change EPSG based on UTM zone
df['lon'], df['lat'] = transformer.transform(df['ig_utm_x'].values, df['ig_utm_y'].values)


In [ ]:
df['ig_date'] = pd.to_datetime(df['ig_date'])

# Function to generate list of event dates
def get_event_dates(row):
    return [row['ig_date'] + pd.Timedelta(days=i) for i in range(int(row['event_dur']))]

df['event_dates'] = df.apply(get_event_dates, axis=1)


In [ ]:
df.head(1)

,id,geometry,ig_date,ig_day,ig_month,ig_year,last_date,event_dur,tot_pix,tot_ar_km2,...,lc_name,lc_desc,lc_type,eco_mode,eco_name,eco_type,tot_perim,lon,lat,event_dates
0,1,MULTIPOLYGON (((-10010798.656358264 4834667.34...,2007-02-01,32,2,2007,2007-02-01 00:00:00,1,1,0.214659,...,Evergreen Needleleaf Forests,Dominated by evergreen conifer trees (canopy>2...,IGBP global vegetation classification scheme,10.0,Central Pacific coastal forests,WWF Terrestrial Ecoregions of the World,1861.250866,163.280081,14.680764,[2007-02-01 00:00:00]


In [ ]:
import requests
import time
import pandas as pd

def get_weather(lat, lon, start_date):
    time.sleep(2)
    url = "https://meteostat.p.rapidapi.com/point/daily"
    headers = {
        "x-rapidapi-key": "d49a6e3da1msh75c4e571ed0ea59p1acda0jsn9a3e8b347dc6"
    }
    params = {
        "lat": lat,
        "lon": lon,
        "start": start_date,
        "end": start_date
    }
    response = requests.get(url, headers=headers, params=params)
    print(response.status_code)
    return response.json()


In [ ]:
def enrich_row(row):
    start = row['ig_date'].split()[0]
    # end = (row['ig_date'] + pd.Timedelta(days=int(row['event_dur']) - 1)).date().isoformat()

    # Get DataFrame of weather
    weather_df = get_weather(row['lat'], row['lon'], start)

    if weather_df.get("data"):
        print(f'getting info for {start}')
        return pd.Series({
            'tavg': weather_df['tavg'],
            'prcp': weather_df['prcp'],
            'wdir': weather_df['wdir'],
            'wspd': weather_df['wspd'],
            'wpgt': weather_df['wpgt'],
            'pres': weather_df['pres'],
            'tsun': weather_df['tsun']
        })




weather_df = df.apply(enrich_row, axis=1)
df = pd.concat([df, weather_df], axis=1)


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


KeyboardInterrupt: 

In [ ]:
df.to_csv('fired_conus_with_weather.csv', index=False)
